<a href="https://colab.research.google.com/github/shumph10/Housing_Assistance_Analysis/blob/sarah_preprocessing/ttest_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import dependencies
import pandas as pd
import numpy as np
import scipy.stats as stats
import mock
from openpyxl.reader import excel

In [2]:
#Read in 2017 & 2022 data sets

#load in files
rent_2017_data = pd.read_excel("sample_data/FY2017_50_rev.xlsx")


#read in values to a dataframe
rent_2017_df = pd.DataFrame(rent_2017_data, columns=["fips2010", "fips2000", "Rent50_2","Rent50_0", "Rent50_1","Rent50_3",
                                                     "Rent50_4", "State","cbsasub","areaname","county","CouSub","countyname",
                                                     "county_town_name", "pop2010", "hu2010", "state_alpha"])
rent_2017_df

,fips2010,fips2000,Rent50_2,Rent50_0,Rent50_1,Rent50_3,Rent50_4,State,cbsasub,areaname,county,CouSub,countyname,county_town_name,pop2010,hu2010,state_alpha
0,100199999,1.002000e+08,886,633,735,1136,1536,1,METRO33860M33860,"Montgomery, AL MSA",1.0,99999,Autauga County,Autauga County,54571.0,22135.0,AL
1,100399999,1.004000e+08,1047,865,908,1524,1751,1,METRO19300M19300,"Daphne-Fairhope-Foley, AL MSA",3.0,99999,Baldwin County,Baldwin County,182265.0,104061.0,AL
2,100599999,1.006000e+08,722,540,543,903,1031,1,NCNTY01005N01005,"Barbour County, AL",5.0,99999,Barbour County,Barbour County,27457.0,11829.0,AL
3,100799999,1.008000e+08,937,719,812,1258,1405,1,METRO13820M13820,"Birmingham-Hoover, AL HUD Metro FMR Area",7.0,99999,Bibb County,Bibb County,22915.0,8981.0,AL
4,100999999,1.010000e+08,937,719,812,1258,1405,1,METRO13820M13820,"Birmingham-Hoover, AL HUD Metro FMR Area",9.0,99999,Blount County,Blount County,57322.0,23887.0,AL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4764,7215199999,7.215200e+09,605,471,502,813,1011,72,METRO41980MM7440,"San Juan-Guaynabo, PR HUD Metro FMR Area",151.0,99999,Yabucoa Municipio,Yabucoa Municipio,37941.0,15798.0,PR
4765,7215399999,7.215400e+09,432,357,374,555,761,72,METRO38660M49500,"Yauco, PR HUD Metro FMR Area",153.0,99999,Yauco Municipio,Yauco Municipio,42043.0,18020.0,PR
4766,7801099999,7.801100e+09,952,764,780,1181,1305,78,NCNTY78010N78010,"St. Croix Island, VI",10.0,99999,St. Croix,St. Croix,53234.0,25275.0,VI
4767,7802099999,7.802100e+09,1529,1038,1230,1897,2096,78,NCNTY78020N78020,"St. John Island, VI",20.0,99999,St. John,St. John,4197.0,3453.0,VI


In [3]:
#deal with datetime value error 
  #something with file generating the wrong type of metadata
  #try, except to get it to run

from datetime import datetime

def datetime_valid(dt_str):
    try:
        datetime.fromisoformat(dt_str.replace('Z', '+00:00'))
    except:
        return False
    return True

In [4]:
#read in with mock to handle ISO datetime error 
filepath = "sample_data/FY2022_FMR_50_county_rev.xlsx"
with mock.patch.object(excel.ExcelReader, 'read_properties', lambda self: None):
  rent_2022_df=pd.read_excel(filepath)

In [5]:
rent_2022_df

,state_code,county_code,county_sub_code,cntyname,town_name,areaname22,rent_50_0,rent_50_1,rent_50_2,rent_50_3,rent_50_4,cbsasub22,pop2017
0,48,59,99999,Callahan County,NaN,"Abilene, TX MSA",715,760,981,1337,1658,METRO10180M10180,13660
1,48,253,99999,Jones County,NaN,"Abilene, TX MSA",715,760,981,1337,1658,METRO10180M10180,19970
2,48,441,99999,Taylor County,NaN,"Abilene, TX MSA",715,760,981,1337,1658,METRO10180M10180,135370
3,72,3,99999,Aguada Municipio,NaN,"Aguadilla-Isabela, PR MSA",407,423,486,604,711,METRO10380M10380,39470
4,72,5,99999,Aguadilla Municipio,NaN,"Aguadilla-Isabela, PR MSA",407,423,486,604,711,METRO10380M10380,55720
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4760,72,133,99999,Santa Isabel Municipio,NaN,Puerto Rico HUD Nonmetro Area,389,396,451,614,674,NCNTY72923N72923,22475
4761,72,147,99999,Vieques Municipio,NaN,Puerto Rico HUD Nonmetro Area,389,396,451,614,674,NCNTY72923N72923,8930
4762,78,10,99999,St. Croix,NaN,"St. Croix Island, VI",964,984,1201,1485,1629,NCNTY78010N78010,53234
4763,78,20,99999,St. John,NaN,"St. John Island, VI",1280,1517,1885,2330,2556,NCNTY78020N78020,4197


In [6]:
#Make a variable for the rent before the pandemic to do stats tests on 
rent_before = rent_2017_df['Rent50_2']
rent_before

0        886
1       1047
2        722
3        937
4        937
        ... 
4764     605
4765     432
4766     952
4767    1529
4768    1093
Name: Rent50_2, Length: 4769, dtype: int64

In [7]:
#Make a variable for the rent after the pandemic to do stats tests on 
rent_after = rent_2022_df['rent_50_2']
rent_after

0        981
1        981
2        981
3        486
4        486
        ... 
4760     451
4761     451
4762    1201
4763    1885
4764    1361
Name: rent_50_2, Length: 4765, dtype: int64

In [8]:
#find the variance for each group
print(np.var(rent_before), np.var(rent_after))

83732.08651166379 165171.53642547037


In [9]:
#perform 2 sample ttest to est if there is a difference in 
  #means before/post pandemic for rent
stats.ttest_ind(a=rent_before, b=rent_after, equal_var=True)

Ttest_indResult(statistic=-21.275211783439826, pvalue=3.6067923796810324e-98)

In [10]:
#get number of null values in 2017 rent dataframe
rent_2017_df.isna().sum(axis=0)

fips2010            0
fips2000            3
Rent50_2            0
Rent50_0            0
Rent50_1            0
Rent50_3            0
Rent50_4            0
State               0
cbsasub             0
areaname            0
county              1
CouSub              0
countyname          0
county_town_name    0
pop2010             2
hu2010              2
state_alpha         0
dtype: int64

In [11]:
#get number of null values in 2022 rent dataframe
rent_2022_df.isnull().sum(axis=0)

state_code            0
county_code           0
county_sub_code       0
cntyname              0
town_name          3158
areaname22            0
rent_50_0             0
rent_50_1             0
rent_50_2             0
rent_50_3             0
rent_50_4             0
cbsasub22             0
pop2017               0
dtype: int64

In [13]:
#drop town_name column 
rent_2022_df.drop(columns=['town_name'], inplace=True)
rent_2022_df

,state_code,county_code,county_sub_code,cntyname,areaname22,rent_50_0,rent_50_1,rent_50_2,rent_50_3,rent_50_4,cbsasub22,pop2017
0,48,59,99999,Callahan County,"Abilene, TX MSA",715,760,981,1337,1658,METRO10180M10180,13660
1,48,253,99999,Jones County,"Abilene, TX MSA",715,760,981,1337,1658,METRO10180M10180,19970
2,48,441,99999,Taylor County,"Abilene, TX MSA",715,760,981,1337,1658,METRO10180M10180,135370
3,72,3,99999,Aguada Municipio,"Aguadilla-Isabela, PR MSA",407,423,486,604,711,METRO10380M10380,39470
4,72,5,99999,Aguadilla Municipio,"Aguadilla-Isabela, PR MSA",407,423,486,604,711,METRO10380M10380,55720
...,...,...,...,...,...,...,...,...,...,...,...,...
4760,72,133,99999,Santa Isabel Municipio,Puerto Rico HUD Nonmetro Area,389,396,451,614,674,NCNTY72923N72923,22475
4761,72,147,99999,Vieques Municipio,Puerto Rico HUD Nonmetro Area,389,396,451,614,674,NCNTY72923N72923,8930
4762,78,10,99999,St. Croix,"St. Croix Island, VI",964,984,1201,1485,1629,NCNTY78010N78010,53234
4763,78,20,99999,St. John,"St. John Island, VI",1280,1517,1885,2330,2556,NCNTY78020N78020,4197


In [14]:
#get number of unique categories in columns
rent_2022_df.nunique()

state_code           56
county_code         329
county_sub_code    1532
cntyname           1961
areaname22         2601
rent_50_0           578
rent_50_1           614
rent_50_2           666
rent_50_3           836
rent_50_4           945
cbsasub22          2601
pop2017            3709
dtype: int64

In [15]:
#get number of unique categories in columns
rent_2017_df.nunique()

fips2010            4769
fips2000            4763
Rent50_2             583
Rent50_0             491
Rent50_1             522
Rent50_3             747
Rent50_4             843
State                 56
cbsasub             2598
areaname            2598
county               329
CouSub              1532
countyname          1961
county_town_name    3175
pop2010             4467
hu2010              4277
state_alpha           56
dtype: int64